# Evaluate Plots

test pair에 대한 iteration을 수행한다.

1. pickle을 통해서 모든 아이템을 다 읽는다.
2. 현재 source sequence를 읽는다.
3. scene similarity를 읽는다.

4. action sequence similarity를 계산한다.
5. activity taxnomy similarity를 계산한다.


다음의 pandas colum을 구성한다. (284x10 정도 되나?)

없는 곳은 None표시를 한다.

source idx, target idx, 
augmentation_id, scene similarity
core activity_gt, core activity_inf
sequence sim, taxonomy sim
sequence bool, taxonomy bool

In [ ]:

import sys
import os
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
import pickle

sys.path.append(os.path.abspath('/root/project')) # add root path to sys.path
sys.path.append(os.path.abspath('/usr/local/lib/python3.10/dist-packages'))
from util import util_constants
from util import util_funcs
import f4_evaluate.evaluate_scene as evaluate_scene
import f1_init.database_init as database_init
import f1_init.agent_init as agent_init
import f1_init.constants_init as constants_init

#Computing similarity
import json
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy.linalg import norm
import ast
import pandas as pd

#PATHS
PATH_CURR_FOLDER = os.path.abspath('') 
PATH_DATA_ANNOTATION = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_annotation'))
PATH_DATA_INPUT_OUTPUT = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output'))
PATH_SOURCE_TARGET_INPUT = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/input/source_target_video_list.pkl'))
TEST_SPATIAL_ANNOTATION_PATH_MANUAL = PATH_DATA_ANNOTATION + '/spatial_testset/manual'

#PAIRSIM DATA
PATH_PAIRSIM = PATH_DATA_ANNOTATION + '/spatial_pairsim_result'

#AUGMENTED DATA PATH
TEST_SPATIAL_ANNOTATION_PATH_SEMI = PATH_DATA_ANNOTATION + '/spatial_testset/semi'    
TEST_SPATIAL_ANNOTATION_PATH_AUG100 = PATH_DATA_ANNOTATION + '/spatial_augmentation/TESTSET_Augmented_Data_100_v3' 
TEST_SPATIAL_ANNOTATION_PATH_AUG67 = PATH_DATA_ANNOTATION + '/spatial_augmentation/TESTSET_Augmented_Data_67_v3' 
TEST_SPATIAL_ANNOTATION_PATH_AUG33 = PATH_DATA_ANNOTATION + '/spatial_augmentation/TESTSET_Augmented_Data_33_v3' 

#BASELINE RESULT PATHS
PATH_BASELINE_RAG = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-rag-0520'))
PATH_BASELINE_NORAG = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-norag-0520'))
PATH_BASELINE_1DIRECT = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-1-direct-0520'))
PATH_BASELINE_1GOALMEDIATION = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-1-goalmediation-0520'))

# PREPROCESS VIDEO_LISTS for TEST DATA & SORT
test_spatial_video_list = database_init.make_spatial_json_video_list(TEST_SPATIAL_ANNOTATION_PATH_MANUAL, TEST_SPATIAL_ANNOTATION_PATH_SEMI)
test_spatial_video_aug100_list = database_init.make_spatial_json_video_list_singlepath(TEST_SPATIAL_ANNOTATION_PATH_AUG100)
test_spatial_video_aug67_list = database_init.make_spatial_json_video_list_singlepath(TEST_SPATIAL_ANNOTATION_PATH_AUG67)
test_spatial_video_aug33_list = database_init.make_spatial_json_video_list_singlepath(TEST_SPATIAL_ANNOTATION_PATH_AUG33)
#Sort test_spatial_video_list first
sorted_test_video_list = sorted(test_spatial_video_list, key=lambda x: x["video_id"])
sorted_test_video_aug33_list = sorted(test_spatial_video_aug33_list, key=lambda x: x["video_id"])
sorted_test_video_aug67_list = sorted(test_spatial_video_aug67_list, key=lambda x: x["video_id"])
sorted_test_video_aug100_list = sorted(test_spatial_video_aug100_list, key=lambda x: x["video_id"])

print(f"augno list len: {len(sorted_test_video_list)}")
print(f"aug100 list len: {len(sorted_test_video_aug33_list)}")
print(f"aug67 list len: {len(sorted_test_video_aug67_list)}")
print(f"aug33 list len: {len(sorted_test_video_aug100_list)}")

# REPAIR GRAPH
def repair_graph(dict_list):
    """
    func: in dict_list, repair faulty graph that only give string rather than dictionalry in the "init_status"
    input: dict_list
    output: dict_list with repaired graphs
    """
    for i in range(len(dict_list)):
        graph = dict_list[i]['spatial_data']
        id = dict_list[i]['video_id']
        for obj in graph:
            init_status = obj.get("init_status")
            
            if isinstance(init_status, str):
                obj["init_status"] = {
                    "status": init_status,
                    "container": None
                }
    return dict_list


sorted_test_video_list = sorted(test_spatial_video_list, key=lambda x: x["video_id"])
sorted_test_video_list= repair_graph(sorted_test_video_list)
sorted_test_video_aug33_list = sorted(test_spatial_video_aug33_list, key=lambda x: x["video_id"])
sorted_test_video_aug67_list = sorted(test_spatial_video_aug67_list, key=lambda x: x["video_id"])
sorted_test_video_aug100_list = sorted(test_spatial_video_aug100_list, key=lambda x: x["video_id"])


# for i in range(len(sorted_test_video_list)):
#     print(sorted_test_video_list[i]['video_id'])

with open(PATH_SOURCE_TARGET_INPUT, "rb") as f:
    source_target_list = pickle.load(f)
    

print(f"source_target_list len: {len(source_target_list)}")
for l in source_target_list:
    print(f"list len in each element: {len(l)}")

print(source_target_list)



In [ ]:
def make_paths(baseline_output_path:str, testset_length:int):
    """
    func: return file path list for baseline folder
    input: baseline output folder absolute path
    output: sorted list[file1, file2, ...]
    """
    print("aaa")

def load_results(baseline_filepaths:list):
    """ 
    func: read and organize results of paths in list
    input: baseline_filepaths
    output: baseline_result_list [[core_activity, source_taxnomy, target_taxnomy, target_sequence, seq_bool, taxo_bool], [], ....]
    """
    baseline_results = []
    for i in range(len(baseline_filepaths)):
        result = []
        core_activty = None
        source_taxnomy = None
        target_taxonomy = None
        target_sequence = None
        seq_bool = None
        taxo_bool = None

        # read files (try)

        # fill each value
        result.append(core_activty)
        result.append(source_taxnomy)
        result.append(target_taxonomy)
        result.append(target_sequence)
        result.append(seq_bool)
        result.append(taxo_bool)
        baseline_results.append(result)
    
    return baseline_results

## Read Sequences and Configure into a List
- make list of augmentation mode
- make list of input sequencs
-----------------------------------
- make list of core activity
- make list of source taxonomy
- make list of common taxonomy
- make list of target taxonomy
- make list of target sequences
- make list of seq bool
- make list of tax bool
-----------------------------------
- make list of similarity between scenes

In [ ]:
# 0. MAKE SOURCE TARGET PAIR CHUNK FOR DIFFERENT AUGMENTATION MODES
source_list = [] # [0,0,0,0,1,1,1,1,...,283] -> [0,1,2,3,..71,0,1,2,3,4..71,...71]
target_list = [] # [augno_0: 55, aug33_0: 33,aug67_0: 28,aug100_0: 22,...] -> [55,26,22,...]

# Since source and target for each augmodes are mixed in element level, need to separate them to different lists.
source_list_augno = [row[0] for row in source_target_list]
source_list_aug33 = [row[0] for row in source_target_list]
source_list_aug67 = [row[0] for row in source_target_list]
source_list_aug100 = [row[0] for row in source_target_list]
source_list.extend(source_list_augno)
source_list.extend(source_list_aug33)
source_list.extend(source_list_aug67)
source_list.extend(source_list_aug100)

target_list_augno = [row[1] for row in source_target_list]
target_list_aug33 = [row[2] for row in source_target_list]
target_list_aug67 = [row[3] for row in source_target_list]
target_list_aug100 = [row[4] for row in source_target_list]
target_list.extend(target_list_augno)
target_list.extend(target_list_aug33)
target_list.extend(target_list_aug67)
target_list.extend(target_list_aug100)
# print(f"len sourceidx targetidx {len(source_list)}, {len(target_list)}")

# 1. make list of sequences
input_sequence_list = []
for i in range(len(source_list)):
    source_action_sequence, source_scene_graph = agent_init.get_video_info(source_list[i])
    input_sequence_list.append(source_action_sequence)

# 2. load files
def load_file(path):
    try:          
         with open(path, "rb") as f:
             data = pickle.load(f)
             return data
    except:
         return None

# For each baselines, save results so that augno->aug33->aug67->aug100 mode chunks are listed sequentially
baseline_paths = [PATH_BASELINE_RAG, PATH_BASELINE_NORAG, PATH_BASELINE_1DIRECT, PATH_BASELINE_1GOALMEDIATION]
baselines = ["rag","norag","1direct","1goalmediation"]
aug_modes = ["augno", "aug33", "aug67", "aug100"]
for baseline_idx, baseline_path in enumerate(baseline_paths):  
    baseline_results = []
    PATH_BASELINE_RESULTS = "evaluation_result/" + baselines[baseline_idx] + ".pkl"
    for i in range(len(source_list)):
            result_dict = {"source_idx":None, "target_idx":None,  baseline": None, "augmode": None, "source_sequence": None, "core_activity": None, "source_taxnomy": None, "common_taxonomy": None, "target_taxonomy": None, "target_sequence": None, "sequence_boolean": None, "taxonomy_boolean": None, "pairwise_similarity": None }
            
            #fill dict without file read
            result_dict["source_idx"] = i
            result_dict["target_idx"]=target_list[i]
            result_dict["baseline"] = baselines[baseline_idx]
            aug_idx = len(aug_modes)*i // len(source_list)
            result_dict["augmode"] = aug_modes[aug_idx]
            result_dict["source_sequence"] = input_sequence_list[i]

            #fill dict with external pickle file
            common_result_path = baseline_path + f"/pair{i}_agent"            
            result_dict["core_activity"] = load_file(common_result_path+"1a.pkl")            
            result_dict["source_taxnomy"] = load_file(common_result_path+"1b.pkl")
            result_dict["common_taxonomy"] = load_file(common_result_path+"2a.pkl")
            result_dict["target_taxonomy"] = load_file(common_result_path+"2b.pkl")
            result_dict["target_sequence"] = load_file(common_result_path+"3.pkl")
            result_agent4= load_file(common_result_path+"4.pkl")
            if(result_agent4 == None):
                result_dict["sequence_boolean"] = None
                result_dict["taxonomy_boolean"] = None       
            else:
                values = [
                    line.split(":")[1].strip()
                    for line in result_agent4.strip().splitlines()
                    if ":" in line
                ]

                result_dict["sequence_boolean"] = True if values[0] == 'yes' else False
                result_dict["taxonomy_boolean"] = True if values[1] == 'yes' else False

        #fill dict with external pairwise scene f1 score
        #augmode, source_idx, target_idx, name in data_annotation/spatial_pairsim_result/comparison_+"modename"+"_idx.pkl"
            file_index = i % len(target_list_augno)
            target_file_index = target_list[i] % len(target_list_augno)

            PATH_PAIRSIM_FILE = PATH_PAIRSIM + f'/comparision_{aug_modes[aug_idx]}_{file_index}.pkl'
            print(f"{file_index}th file for {aug_modes[aug_idx]} mode, target:{target_file_index}")

            with open(PATH_PAIRSIM_FILE, "rb") as f:
                similarity_data = pickle.load(f)                
                entity_result = similarity_data[target_file_index]['entity_result']
                f1_score = entity_result["f1_score"]
                print(f1_score)
                result_dict["pairwise_similarity"] = f1_score

        #append dict for each baseline
            baseline_results.append(result_dict)
    
    #3. save baseline results into a single pickle file
    if os.path.exists(PATH_BASELINE_RESULTS):
        os.remove(PATH_BASELINE_RESULTS)            
        print(f"removed existing file for {PATH_BASELINE_RESULTS}")

    with open(PATH_BASELINE_RESULTS, 'wb') as f:
        pickle.dump(baseline_results, f)
        print(f"saved {baselines[baseline_idx]} file at {PATH_BASELINE_RESULTS}")
         


## Perform Numerical Calculation of Results

### base results
- pairwise scene similarity
- seq2seq similarity
- seq2seq taxonomy similarity
### plot
- per scenario similarity analysis
- per augmentation, per scenario similarity analysis
- per scenario similarity vs similarity



In [ ]:
# baseline paths
baselines = ["rag","norag","1direct","1goalmediation"]
aug_modes = ["augno", "aug33", "aug67", "aug100"]
baseline_results_paths = []
for idx in range(len(baselines)):
    path = "evaluation_result/" + baselines[idx] + ".pkl"
    baseline_results_paths.append(path)
print(baseline_results_paths)

# load baselines
baseline_rag = []
baseline_norag = []
baseline_1direct = []
baseline_1goalmediation = []
with open(baseline_results_paths[0], "rb") as f:
    baseline_rag = pickle.load(f)
with open(baseline_results_paths[1], "rb") as f:
    baseline_norag = pickle.load(f)
with open(baseline_results_paths[2], "rb") as f:
    baseline_1direct = pickle.load(f)
with open(baseline_results_paths[3], "rb") as f:
    baseline_1goalmediation = pickle.load(f)            
print(f"baseline data length {len(baseline_rag)}, {len(baseline_norag)}, {len(baseline_1direct)}, {len(baseline_1goalmediation)}")

In [ ]:
# define functions for evaluation
def calculate_evaluation(baseline_result, path, model):
    """
    func: calculate (similarities) for baseline result items
    input: result for one baseline [{dict3}{dict2}..{dictN}]
    result: 
    """
    dict_list = []
    for idx, dict in enumerate(baseline_result):
        evaluation_dict = {"source_idx":None, "target_idx":None, "pairwise_similarity":None,"sequence_similarity":None, "taxonomy_similarity":None, "sequence_boolean":False, "taxonomy_boolean": False}
        print(f"{idx}th result")
        sequence_similarity, taxonomy_similarity = compute_similarities(dict, model)
        print(f"{sequence_similarity} {taxonomy_similarity}")
        evaluation_dict["source_idx"]= dict['source_idx']
        evaluation_dict["target_idx"]= dict['target_idx']
        evaluation_dict["pairwise_similarity"]= dict['pairwise_similarity']
        evaluation_dict["sequence_similarity"]= sequence_similarity
        evaluation_dict["taxonomy_similarity"]= taxonomy_similarity
        evaluation_dict["sequence_boolean"]= dict['sequence_boolean']
        evaluation_dict["taxonomy_boolean"]= dict['taxonomy_boolean']
        dict_list.append(evaluation_dict)
    df = pd.DataFrame(dict_list)
    with open(path, 'wb') as f:
        pickle.dump(df, f)
        print(f"saved baseline evaluation as dataframe")    
    return df    

def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    if norm(vec1) == 0 or norm(vec2) == 0:
        return 0.0
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

def safe_parse_sequence(seq):
    if seq is None:
        return []
    if isinstance(seq, list):
        return seq
    try:
        return ast.literal_eval(seq)
    except (ValueError, SyntaxError):
        return []

def safe_parse_taxonomy(tax):
    if tax is None:
        return {}
    if isinstance(tax, dict):
        return tax
    try:
        return json.loads(tax)
    except (json.JSONDecodeError, TypeError):
        return {}

def compute_similarities(entry, embed_model):
    # --- Parse sequences ---
    source_seq = safe_parse_sequence(entry.get('source_sequence'))
    target_seq = safe_parse_sequence(entry.get('target_sequence'))

    if not source_seq or not target_seq:
        seq_similarity = 0.0
    else:
        source_seq_str = ' '.join(source_seq)
        target_seq_str = ' '.join(target_seq)
        source_seq_emb = embed_model.encode(source_seq_str)
        target_seq_emb = embed_model.encode(target_seq_str)
        seq_similarity = cosine_similarity(source_seq_emb, target_seq_emb)

    # --- Parse taxonomies ---
    source_tax = safe_parse_taxonomy(entry.get('source_taxnomy'))
    target_tax = safe_parse_taxonomy(entry.get('target_taxonomy'))

    if not source_tax or not target_tax:
        tax_similarity = 0.0
    else:
        source_tax_str = ' '.join(f"{k}: {v}" for k, v in source_tax.items())
        target_tax_str = ' '.join(f"{k}: {v}" for k, v in target_tax.items())
        source_tax_emb = embed_model.encode(source_tax_str)
        target_tax_emb = embed_model.encode(target_tax_str)
        tax_similarity = cosine_similarity(source_tax_emb, target_tax_emb)

    return seq_similarity, tax_similarity




# Compute
baselines = ["rag","norag","1direct","1goalmediation"]
baseline_df_paths = []
for idx in range(len(baselines)):
    path = "evaluation_result/" + baselines[idx] + "_df" +".pkl"
    baseline_df_paths.append(path)


print(baseline_df_paths)
model = SentenceTransformer('all-MiniLM-L6-v2')

df_rag = calculate_evaluation(baseline_rag, baseline_df_paths[0], model)
df_norag = calculate_evaluation(baseline_norag, baseline_df_paths[1], model)
df_1direct = calculate_evaluation(baseline_1direct, baseline_df_paths[2], model)
df_1goalmediation = calculate_evaluation(baseline_1goalmediation, baseline_df_paths[3], model)



In [ ]:
# run evaluation functions
import pandas as pd
import matplotlib.pyplot as plt

# Assume df is your DataFrame

# Step 1: Filter where taxonomy_boolean is True
def plot_taxonomy(df):
    filtered_df = df[df['taxonomy_boolean'] == True]
    # filtered_df = df

    # Step 2: Plot
    plt.figure(figsize=(8, 6))
    plt.scatter(filtered_df['pairwise_similarity'], filtered_df['taxonomy_similarity'], alpha=0.7)
    plt.xlabel('Pairwise Similarity')
    plt.ylabel('Taxonomy Similarity')
    plt.title('Pairwise vs. Taxonomy Similarity (Only where taxonomy_boolean=True)')

    plt.xlim(0, 0.3)
    plt.ylim(-1, 1)

    plt.grid(True)
    plt.tight_layout()
    plt.show()


def plot_sequence(df):
    filtered_df = df[df['taxonomy_boolean'] == True]
    filtered_df = filtered_df[filtered_df['sequence_boolean'] == True]

    # filtered_df = df

    # Step 2: Plot
    plt.figure(figsize=(8, 6))
    plt.scatter(filtered_df['pairwise_similarity'], filtered_df['sequence_similarity'], alpha=0.7)
    plt.xlabel('Pairwise Similarity')
    plt.ylabel('Sequence Similarity')
    plt.title('Pairwise vs. Sequence Similarity (Only where sequence_boolean=True)')

    plt.xlim(0, 0.3)
    plt.ylim(-0.3, 0.3)  

    plt.grid(True)
    plt.tight_layout()
    plt.show()    

plot_taxonomy(df_rag)
plot_taxonomy(df_norag)
plot_taxonomy(df_1direct)
plot_taxonomy(df_1goalmediation)

plot_sequence(df_rag)
plot_sequence(df_norag)
plot_sequence(df_1direct)
plot_sequence(df_1goalmediation)
